# Generator

In [9]:
def printf(nums):
    for num in nums:
        print('num is %d'%num)
        yield num
def adder(nums):
    for num in nums:
        print('adding %d'%num)
        if num % 2 == 0:
            yield num + 1
        else :
            yield num + 2

In [10]:
nums = [1, 2, 3, 4,5,6,7,8]

In [11]:
result = adder(printf(nums))

In [12]:
next(result)

num is 1
adding 1


3

In [53]:
def fab(n):
    i, a, b = 0, 0, 1
    while i <= n:
        yield b
        i = i+ 1
        a, b = b, a+b

In [54]:
for i in fab(10):
    print(i)

1
1
2
3
5
8
13
21
34
55
89


# Generator 与代码交互

In [1]:
def psychologlist():
    print('Please tell me you problems')
    while True:
        answer = (yield)
        if answer is not None:
            if answer.endswith('?'):
                print('do not ask your self to much questions')
            elif 'good' in answer:
                print("Ahh , that's good , go on")
            elif 'bad' in answer:
                print("Don't be so negative")

In [2]:
free = psychologlist()
next(free)

Please tell me you problems


In [3]:
free.send('what the hell?')

do not ask your self to much questions


In [4]:
free.send('today is fucking bad day')

Don't be so negative


In [5]:
free.send('today is fucking good day')

Ahh , that's good , go on


# Decorator

- 代码更简洁，易读
- 不能使用lambda
- 被装饰的函数被调用时，接受单一参数，返回可调用对象（callable）

In [6]:
class WithoutDecorators:
    def some_static_method():
        print('this is a static method')
    # 类的静态方法
    some_static_method = staticmethod(some_static_method)
    def some_class_method(cls):
        print('this is as class method')
    some_class_method = classmethod(some_class_method)
    
class WithDecorators:
    @staticmethod
    def some_static_method():
        print('this is a static method')
    @classmethod
    def some_class_method():
        print('this is a class method')

```python
def mydecorator(function):
    def wrapped(*args, **kwargs):
        # 在调用原始函数之前，做些什么
        result = function(*args, **kwargs)
        # 函数调用之后，做些什么
        # 返回结果
        return result 
    # 返回装饰后的函数
    return wrapped
```

#### 非参数化装饰器类
```python
class DecoratorAsClass:
    def __init__(self, function):
        self.function = function
    def __call__(self, *args, **kwargs):
        # 调用原始函数前
        result = self.function(*args, **kwargs)
        # 调用函数之后
        # 返回结果
        return result
```

#### 参数化装饰器， 需要用到第二层包装
```python
def repeat(num=3):
    # num 装饰器参数
    def actual_decorator(function):
        def wrapper(*args, **kwargs):
            result = None
            for _ in range(num):
                result = function(*args, **kwargs)
            return result
        return wrapper
    return actual_decorator


@repeat(3)   # 带参数的装饰器，即便是默认参数也要带()
def foo():
    print('today is fucking bad day')
    
foo()

>>>today is fucking bad day
>>>today is fucking bad day
>>>today is fucking bad day
```

#### 装饰器的内省情况：
&emsp;&emsp;使用装饰器时不保存函数的元数据(函数的注释文档和原始函数名),装饰器创建了一个新函数，并返回它，完全没有考虑被装饰函数的标识，所以在调试被装饰过的函数时会很麻烦，不能访问原始的文档字符串和函数签名。

```python
def dummy_decorator(function):
    def wrapped(*args, **kwargs):
        '''包装函数的内部文档'''
        return function(*args, **kwargs)
    return wrapped

@dummy_decorator
def function_a():
    '''被装饰函数的内部文档'''
    pass


>>>function_a.__name__
>>>'wrapped'
>>>function_a.__doc__
>>>'包装函数的文档'
```

#### 如何解决装饰器的内省问题

- `form functools import wraps`

```python
from functools import wraps

def preserving_decorator(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        return function(*args, **kwargs)
    return wrapped

@preserving_decorator
def function_b():
    '''被装饰函数的内部文档'''
    pass

>>>function_b.__name__
>>>'function_b'
>>>function_b.__doc__
>>>'被装饰函数的内部文档'
```